# Finding Similar Shoes Using Vector Search in Db2

## Prepare DB2

Ensure DB2 is up and running.  The following commands in the DB2 container should help:

```bash
su - db2inst1
export PATH=/opt/ibm/db2/V12.1/bin:$PATH
db2start
db2 create database testdb
db2 list database directory
```

In [ ]:
import pandas as pd
import os
from dotenv import dotenv_values, load_dotenv
from ibm_watsonx_ai import APIClient, Credentials
from ibm_watsonx_ai.foundation_models import Embeddings
from utils import *
import ibm_db

pd.set_option('display.max_colwidth', None)  # Show full column content

## Setting up wx.ai API for embedding generation

In [ ]:
load_dotenv(os.getcwd()+"/.env", override=True)

credentials = Credentials(
               url = os.getenv("WATSONX_URL", "https://us-south.ml.cloud.ibm.com"),
               api_key = os.getenv("WATSONX_APIKEY", "")
               )

client = APIClient(credentials)

project_id = os.getenv("WATSONX_PROJECT", "")
client.set.default_project(project_id)

embeddings = Embeddings(
   model_id=client.foundation_models.EmbeddingModels.MULTILINGUAL_E5_LARGE,
   credentials=credentials,
   project_id=project_id,
)

## Setting up Db2 Connection

In [ ]:
dbName = os.getenv('database')
dbHost = os.getenv('hostname')
dbPort = os.getenv('port')
dbUser = os.getenv('uid')
dbPassword = os.getenv('pwd')

connection = f"DATABASE={dbName};HOSTNAME={dbHost};PORT={dbPort};PROTOCOL=TCPIP;UID={dbUser};PWD={dbPassword};"
connOption = {ibm_db.SQL_ATTR_AUTOCOMMIT : ibm_db.SQL_AUTOCOMMIT_ON}
conn = ibm_db.connect(connection, "", "", connOption)

if conn is None:
    print("\nERROR: Unable to connect to the \'" + dbName + "\' database.")
    ibm_db.close(conn)
    jupyter.notebook.stop('shoes-search.ipynb')
else:
    print("\nSUCCESS: Connected to the \'" + dbName + "\' database")

# Setting up a Shoes Table at Db2

In [ ]:
# Drop the DB table SQ_SHOES if it exists

sqlStatement1 = "DROP TABLE SQ_SHOES"
try:
    resultSet1 = ibm_db.exec_immediate(conn, sqlStatement1)
except Exception:
    pass

In [ ]:
# Create the table
sqlStatement2 = """
    CREATE TABLE SQ_SHOES (
    SKU VARCHAR(8),
    PRODUCT_NAME VARCHAR(23),
    BRAND VARCHAR(9),
    CLASS VARCHAR(5),
    TYPE VARCHAR(7),
    MATERIAL VARCHAR(9),
    COLOR VARCHAR(5),
    WEATHER_RESISTANCE VARCHAR(10),
    ARCH_SUPPORT VARCHAR(4),
    SIZE FLOAT,
    PRICE FLOAT,
    RATING FLOAT,
    STORE_ID BIGINT,
    CITY VARCHAR(7),
    EMBEDDING VECTOR(1024,FLOAT32)
    );
    """
try:
    resultSet2 = ibm_db.exec_immediate(conn, sqlStatement2)
except Exception:
    pass

if resultSet2 is False:
    print("\nERROR: Unable to create table SQ_SHOES\n")
    ibm_db.close(conn)
    jupyter.notebook.stop('shoes-search.ipynb')
else:
    print("\nSUCESSSL Created table SQ_SHOES\n")

In [ ]:
# Load shoes-vectors spreadsheet into the database

shoes = pd.read_csv('shoes-vectors.csv')

sql_insert = "INSERT INTO SQ_SHOES(SKU,PRODUCT_NAME,BRAND,CLASS,TYPE,MATERIAL,COLOR,WEATHER_RESISTANCE,ARCH_SUPPORT,SIZE,PRICE,RATING,STORE_ID,CITY,EMBEDDING) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,VECTOR(?, 1024, FLOAT32))"

preparedStmt = None
try:
    preparedStmt = ibm_db.prepare(conn, sql_insert)
except Exception:
    print(Exception)
  
if preparedStmt is False:
    print("\nERROR: Unable to prepare the SQL statement specified.\n")
    ibm_db.close(conn)
else:
  tuple_of_tuples = tuple([tuple(x) for x in shoes.values])
  print(tuple_of_tuples)

try:
  ibm_db.execute_many(preparedStmt, tuple(tuple_of_tuples))
except Exception:
  print(Exception)
  

## Searching for a Running Shoe of Size `12`, for `Men`

In [ ]:
sqlStatement3 = "SELECT count(*) FROM SQ_SHOES"
try:
    resultSet3 = ibm_db.exec_immediate(conn, sqlStatement3)
except Exception:
    pass

if resultSet3 is False:
    print("\nERROR: Unable to create table SQ_SHOES\n")
    ibm_db.close(conn)
    jupyter.notebook.stop('shoes-search.ipynb')
else:
    while (ibm_db.fetch_row(resultSet3) is True):
      print(ibm_db.result(resultSet3, 0))
    print()

In [ ]:
def db2_resultSet_to_dataFrame(resultSet) -> pd.DataFrame:
  ret = None
  data = []
  if resultSet is False:
      print("\nERROR: Unable to fetch mens shoes\n")
      ibm_db.close(conn)
  else:
      # Fetch column names from result
      colNames=[]
      numColumns = ibm_db.num_fields(resultSet)
      for counter in range(0, numColumns):
        colNames.append(ibm_db.field_name(resultSet, counter))
      # Fetch rows as tuples and apend to array
      noData = False
      while noData is False:
        dataRow = ibm_db.fetch_tuple(resultSet)
        if dataRow is False:
          noData = True
        else:
          data.append(dataRow)
          
      # convert to pandas dataframe
      ret = pd.DataFrame(data, columns=colNames)
  return ret

In [ ]:
sqlStatement4 = """ 
    SELECT SKU, PRODUCT_NAME, BRAND, TYPE, MATERIAL, COLOR, WEATHER_RESISTANCE, ARCH_SUPPORT, PRICE, RATING, CITY
    FROM SQ_SHOES 
    WHERE CLASS = 'Men' AND Size = 12 
    FETCH FIRST 6 ROWS ONLY
    """

try:
    resultSet4 = ibm_db.exec_immediate(conn, sqlStatement4)
except Exception:
    pass

if resultSet4:        
    df_shoe_search = db2_resultSet_to_dataFrame(resultSet4)

    # display selected shoes
    sku_list = df_shoe_search['SKU'].tolist()
    display_sku_images(sku_list)

    print(df_shoe_search.head(6))

### My chosen shoe is available at the Ottawa location

In [ ]:
my_choice_sku = 'ZEN-2061'

## Searching for a similar shoes at the Toronto Location

In [ ]:
sqlStatement5 = f"""
    SELECT SKU, PRODUCT_NAME, BRAND, TYPE, MATERIAL, COLOR, WEATHER_RESISTANCE, ARCH_SUPPORT, PRICE, RATING,
        VECTOR_DISTANCE((SELECT EMBEDDING FROM SQ_SHOES WHERE SKU = '{my_choice_sku}'), EMBEDDING, EUCLIDEAN) AS DISTANCE
    FROM SQ_SHOES
    WHERE SKU <> '{my_choice_sku}' AND CITY = 'Toronto' AND SIZE = 12 AND CLASS = 'Men'
    ORDER BY DISTANCE ASC
    FETCH FIRST 3 ROWS ONLY
    """

try:
    resultSet5 = ibm_db.exec_immediate(conn, sqlStatement5)
except Exception:
    pass

if resultSet5:        
    df_shoes_results = db2_resultSet_to_dataFrame(resultSet5)
    sku_list = df_shoes_results['SKU'].tolist()

    display_sku_images(sku_list)
    df_shoes_results.head(3)

## Comparing the Search Results with my Preferred Shoe

In [ ]:
sku_list = df_shoes_results['SKU'].tolist()
sku_list.append(my_choice_sku)

sku_sql_in = ', '.join(f"'{sku}'" for sku in sku_list)

sqlStatement6 = f"""
    SELECT SKU, PRODUCT_NAME, BRAND, TYPE, MATERIAL, COLOR, WEATHER_RESISTANCE, ARCH_SUPPORT, PRICE, RATING,
        VECTOR_DISTANCE((SELECT EMBEDDING FROM SQ_SHOES WHERE SKU = '{my_choice_sku}'), EMBEDDING, EUCLIDEAN) AS DISTANCE
    FROM SQ_SHOES
    WHERE SKU IN ({sku_sql_in})
    ORDER BY DISTANCE ASC
    """
try:
    resultSet6 = ibm_db.exec_immediate(conn, sqlStatement6)
except Exception:
    pass

if resultSet6:        
    df_result = db2_resultSet_to_dataFrame(resultSet6)

cols_show = ['SKU', 'TYPE', 'MATERIAL', 'COLOR', 'WEATHER_RESISTANCE', 'ARCH_SUPPORT']
df_result[cols_show].transpose()

## Looking under the hood

### 1. original table without a `VECTOR` column

In [ ]:
# Drop the DB table SQ_SHOES if it exists

sqlStatement1 = "DROP TABLE SQ_SHOES"
try:
    resultSet1 = ibm_db.exec_immediate(conn, sqlStatement1)
except Exception:
    pass

In [ ]:
# Create the table
sqlStatement2 = """
    CREATE TABLE SQ_SHOES (
        SKU VARCHAR(8),
        PRODUCT_NAME VARCHAR(23),
        BRAND VARCHAR(9),
        CLASS VARCHAR(5),
        TYPE VARCHAR(7),
        MATERIAL VARCHAR(9),
        COLOR VARCHAR(5),
        WEATHER_RESISTANCE VARCHAR(10),
        ARCH_SUPPORT VARCHAR(4),
        SIZE FLOAT,
        PRICE FLOAT,
        RATING FLOAT,
        STORE_ID BIGINT,
        CITY VARCHAR(7)
    );
    """

try:
    resultSet2 = ibm_db.exec_immediate(conn, sqlStatement2)
except Exception:
    pass

if resultSet2 is False:
    print("\nERROR: Unable to create table SQ_SHOES\n")
    ibm_db.close(conn)
    jupyter.notebook.stop('shoes-search.ipynb')
else:
    print("\nSUCESSSL Created table SQ_SHOES\n")

In [ ]:
# Load shoes-vectors spreadsheet into the database

shoes = pd.read_csv('shoes.csv')

sql_insert = "INSERT INTO SQ_SHOES(SKU,PRODUCT_NAME,BRAND,CLASS,TYPE,MATERIAL,COLOR,WEATHER_RESISTANCE,ARCH_SUPPORT,SIZE,PRICE,RATING,STORE_ID,CITY) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)"

preparedStmt = None
try:
    preparedStmt = ibm_db.prepare(conn, sql_insert)
except Exception:
    print(Exception)
  
if preparedStmt is False:
    print("\nERROR: Unable to prepare the SQL statement specified.\n")
    ibm_db.close(conn)
else:
  tuple_of_tuples = tuple([tuple(x) for x in shoes.values])
  print(tuple_of_tuples)

try:
  ibm_db.execute_many(preparedStmt, tuple(tuple_of_tuples))
except Exception:
  print(Exception)



In [ ]:
# Print first few rows of SQ_SHOE table

sqlStatement3 = "SELECT * FROM SQ_SHOES FETCH FIRST 3 ROWS ONLY"

try:
    resultSet3 = ibm_db.exec_immediate(conn, sqlStatement3)
except Exception:
    pass

if resultSet3:        
    df_shoes = db2_resultSet_to_dataFrame(resultSet3)
    print(df_shoes)

### 2. Adding a `VECTOR` column

In [ ]:
sqlStatement4 = "ALTER TABLE SQ_SHOES ADD COLUMN EMBEDDING VECTOR(1024, FLOAT32);"

try:
    resultSet4 = ibm_db.exec_immediate(conn, sqlStatement4)
except Exception:
    pass

if resultSet4 is False:
    print("\nERROR: Unable to alter table SQ_SHOES\n")
    ibm_db.close(conn)
    jupyter.notebook.stop('shoes-search.ipynb')
else:
    print("\nSUCESSSL Altered table SQ_SHOES\n")

### 1. selected shoe features for generating shoe vectors

In [ ]:
embedding_cols = ['TYPE', 'MATERIAL', 'COLOR', 'WEATHER_RESISTANCE', 'ARCH_SUPPORT']
sqlStatement5 = f"SELECT * FROM SQ_SHOES WHERE SKU = '{my_choice_sku}'"

try:
    resultSet5 = ibm_db.exec_immediate(conn, sqlStatement5)
except Exception:
    pass

shoe_sample = None
if resultSet5:        
    shoe_sample = db2_resultSet_to_dataFrame(resultSet5)

shoe_sample[embedding_cols]

### 2. combined the text features of each shoe as follows

In [ ]:
# Combine all columns into a single string for each row, including column names
shoe_sample['combined'] = shoe_sample.apply(
    lambda row: ' [SEP] '.join([f"{col_name}: {row[col_name]}" for col_name in embedding_cols]), 
    axis=1
)

shoe_sample['combined']

### 3. generated embedding vectors for the shoe text obtained in the previous step. For generating embedding vectors, used `watsonx.ai`

In [ ]:

row_combined = shoe_sample['combined'].tolist()
print('input text: ', row_combined[0])

shoe_vectors = embeddings.embed_documents(texts=row_combined)
print('generated embedding vector: ', shoe_vectors[0])

shoe_sample['embedding'] = shoe_vectors
shoe_sample['embedding'] = shoe_sample['embedding'].apply(lambda x: '[' + ', '.join(map(str, x)) + ']')

### 4. stored vectors in Db2 in a Vector column

In [ ]:
sqlStatement6 = f"SELECT EMBEDDING FROM SQ_SHOES WHERE SKU = '{my_choice_sku}'"

try:
    resultSet6 = ibm_db.exec_immediate(conn, sqlStatement6)
except Exception:
    pass

shoe = None
if resultSet6:        
    shoe = db2_resultSet_to_dataFrame(resultSet6)
    print(shoe)

In [ ]:
update_sql = f""" 
UPDATE SQ_SHOES SET EMBEDDING = VECTOR('{shoe_vectors[0]}', 1024, FLOAT32) WHERE SKU = '{my_choice_sku}'
"""

try:
    resultSet7 = ibm_db.exec_immediate(conn, update_sql)
except Exception:
    pass


In [ ]:
sqlStatement8 = f"SELECT EMBEDDING FROM SQ_SHOES WHERE SKU = '{my_choice_sku}'"

try:
    resultSet8 = ibm_db.exec_immediate(conn, sqlStatement8)
except Exception:
    pass

shoe = None
if resultSet8:        
    shoe = db2_resultSet_to_dataFrame(resultSet8)
    print(shoe)

### 5. Loading the vectors of all the shoes - to save demo time

To save time the table will be reloaded from the csv file containing the vactors rather than calling out to watsonx.ai to generate and store all the vectors.

In [ ]:
# Drop the DB table SQ_SHOES

sqlStatement9 = "DROP TABLE SQ_SHOES"
try:
    resultSet9 = ibm_db.exec_immediate(conn, sqlStatement9)
except Exception:
    pass

In [ ]:
# Create the table
sqlStatement10 = """
    CREATE TABLE SQ_SHOES (
    SKU VARCHAR(8),
    PRODUCT_NAME VARCHAR(23),
    BRAND VARCHAR(9),
    CLASS VARCHAR(5),
    TYPE VARCHAR(7),
    MATERIAL VARCHAR(9),
    COLOR VARCHAR(5),
    WEATHER_RESISTANCE VARCHAR(10),
    ARCH_SUPPORT VARCHAR(4),
    SIZE FLOAT,
    PRICE FLOAT,
    RATING FLOAT,
    STORE_ID BIGINT,
    CITY VARCHAR(7),
    EMBEDDING VECTOR(1024,FLOAT32)
    );
    """
try:
    resultSet10 = ibm_db.exec_immediate(conn, sqlStatement10)
except Exception:
    pass

if resultSet10 is False:
    print("\nERROR: Unable to create table SQ_SHOES\n")
    ibm_db.close(conn)
    jupyter.notebook.stop('shoes-search.ipynb')
else:
    print("\nSUCESSSL Created table SQ_SHOES\n")

In [ ]:
# Load shoes-vectors spreadsheet into the database

shoes = pd.read_csv('shoes-vectors.csv')

sql_insert = "INSERT INTO SQ_SHOES(SKU,PRODUCT_NAME,BRAND,CLASS,TYPE,MATERIAL,COLOR,WEATHER_RESISTANCE,ARCH_SUPPORT,SIZE,PRICE,RATING,STORE_ID,CITY,EMBEDDING) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,VECTOR(?, 1024, FLOAT32))"

preparedStmt = None
try:
    preparedStmt = ibm_db.prepare(conn, sql_insert)
except Exception:
    print(Exception)
  
if preparedStmt is False:
    print("\nERROR: Unable to prepare the SQL statement specified.\n")
    ibm_db.close(conn)
else:
  tuple_of_tuples = tuple([tuple(x) for x in shoes.values])
  print(tuple_of_tuples)

try:
  ibm_db.execute_many(preparedStmt, tuple(tuple_of_tuples))
except Exception:
  print(Exception)


### 6. Finding matching shoes using the `VECTOR_DISTANCE` function

In [ ]:
sqlStatement11 = f"""
SELECT 
    SKU, 
    PRODUCT_NAME, 
    BRAND, 
    TYPE, 
    MATERIAL, 
    COLOR, 
    WEATHER_RESISTANCE, 
    ARCH_SUPPORT, 
    PRICE, 
    RATING,
    VECTOR_DISTANCE(
        (SELECT EMBEDDING FROM SQ_SHOES WHERE SKU = '{my_choice_sku}'), 
        EMBEDDING, 
        EUCLIDEAN
    ) AS DISTANCE
FROM 
    SQ_SHOES
WHERE 
    SKU <> '{my_choice_sku}'
    AND CITY = 'Toronto'
    AND SIZE = 12
    AND CLASS = 'Men'
ORDER BY 
    DISTANCE ASC
FETCH FIRST 5 ROWS ONLY
"""

try:
    resultSet11 = ibm_db.exec_immediate(conn, sqlStatement11)
except Exception:
    pass

top_shoes = None
if resultSet11:        
    top_shoes = db2_resultSet_to_dataFrame(resultSet11)
    print(top_shoes.head())



### 7. Visualizing vector search

In [ ]:

sku_list = top_shoes['SKU'].tolist()
sku_list.append(my_choice_sku)
sku_sql_in = ', '.join(f"'{sku}'" for sku in sku_list)
sqlStatement12=f"""
    SELECT 
        SKU, 
        PRODUCT_NAME, 
        VECTOR_DISTANCE(
            (SELECT EMBEDDING FROM SQ_SHOES WHERE SKU = '{my_choice_sku}'), 
            EMBEDDING, 
            EUCLIDEAN
        ) AS DISTANCE,
        VECTOR_SERIALIZE(EMBEDDING) AS EMBEDDING 
    FROM 
        SQ_SHOES
    WHERE 
        SKU IN ({sku_sql_in})
    """

try:
    resultSet12 = ibm_db.exec_immediate(conn, sqlStatement12)
except Exception:
    pass

top_matching_vectors = None
if resultSet12:        
    top_matching_vectors = db2_resultSet_to_dataFrame(resultSet12)

plot_similarity_tsne(top_matching_vectors, my_choice_sku)

In [ ]:
# Tidy up after demo - Drop the DB table SQ_SHOES

sqlStatement13 = "DROP TABLE SQ_SHOES"
try:
    resultSet13 = ibm_db.exec_immediate(conn, sqlStatement13)
except Exception:
    pass

In [ ]:
# Close the connection to DB2
if not conn is None:
    print("Disconnecting from the \'" + dbName + "\' database ... ", end="")
    try:
        returnCode = ibm_db.close(conn)
    except Exception:
        pass